# Quickstart tutorial

Welcome to the quickstart tutorial to Esmraldi! If you have comments or suggestions, please don’t hesitate to reach out. 
This tutorial contains the key steps of the imaging processing workflow leading to the joint analysis of a MALDI image with another image. It uses a basic MALDI dataset along with a synthetic image. 

## Input, Visualization

The data is available in the `data` directory. 
First, read the data and display the spectrum of the first pixel:

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
rootpath = "../../"
sys.path.append(os.path.join(os.path.dirname(os.path.abspath("__file__")), rootpath))

import esmraldi.imzmlio as io
import matplotlib.pyplot as plt

imzml = io.open_imzml(rootpath + "data/Example_Continuous.imzML")
spectra = io.get_spectra(imzml)

print(spectra.shape)
mz_first, intensities_first = spectra[0, 0], spectra[0, 1]
plt.plot(mz_first, intensities_first)

The variable `spectra` is a 3D numpy array, where the first dimension corresponds to pixels, the second dimension to *m/z* or intensities, and the last dimension to their associated values. Here, this array contains 9 pixels, and each spectrum has 8399 points.

Then, we display the image of the ion of *m/z* 328.9 +/- 0.25:

In [ ]:
image = io.get_image(imzml, 328.9, 0.25)
plt.imshow(image)

The image is a 3x3 array of various intensities.

## Spectra processing

### Peak detection

The next step is to detect peaks across all spectra. Our approach is based on [topographical prominence](https://en.wikipedia.org/wiki/Topographic_prominence) values. More specifically, we define the **local** prominence as the ratio between the prominence and the estimated local noise in the signal. The local noise $\sigma$ is estimated as the median absolute deviations in a window of length $w$. Let $f$ be the local prominence factor, the local maxima whose intensity is above $f * \sigma$ are considered as peaks.

In [ ]:
import esmraldi.spectraprocessing as sp
import numpy as np

prominence_local_factor = 4
window_length = 1000

detected_mzs = sp.spectra_peak_mzs_adaptative(spectra, factor=prominence_local_factor, wlen=window_length)
detected_mz_first = detected_mzs[0]
indices = np.in1d(mz_first, detected_mz_first)
detected_intensities_first = intensities_first[indices]

plt.plot(mz_first, intensities_first, detected_mz_first, detected_intensities_first, "o")

The original spectrum is displayed in blue, and the detected peaks are highlighted by orange dots.


### Spectra alignment
Next, the spectra need to be realigned to compensate for *m/z* differences between pixels, due to instrumentation differences, or sample inhomogeneity.

Groups of close *m/z* values are made with a tolerance given by the *step* parameter. The median *m/z* value in each group is taken as reference for the alignment procedure.

In [ ]:
realigned_spectra = sp.realign_mzs(spectra, detected_mzs, reference="median", nb_occurrence=1, step=0.05)
print(realigned_spectra.shape)

After the realignment, 121 peaks are detected (vs. the initial 8399 peaks).

## Segmentation

Next, a representative image is extracted from the set of MALDI ion images. This is achieved by regio growing on a subset of relevant images, that is to say non-noisy images.

Relevant images are found by a measure called *spatial coherence* which considers the minimum area of the largest connected component in ion binary images obtained from a set of quantile thresholds.

In [ ]:
import esmraldi.segmentation as seg

maldi_image = io.get_images_from_spectra(realigned_spectra, (3,3))
maldi_image = io.normalize(maldi_image)
# Relevant images
spatially_coherent = seg.find_similar_images_area(maldi_image, 0, quantiles=[60, 70, 80, 90])

# Mean image
mean_image = np.average(spatially_coherent, axis=-1)

# Region growing
seeds = set(((1, 1), (0, 0)))
list_end, _ = seg.region_growing(spatially_coherent, seedList=seeds, lower_threshold=50)
x = [elem[0] for elem in list_end]
y = [elem[1] for elem in list_end]
mask = np.ones_like(mean_image)
mask[x, y] = 0
masked_mean_image = np.ma.array(mean_image, mask=mask)
masked_mean_image = masked_mean_image.filled(0)

plt.imshow(masked_mean_image)

## Registration

